In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import AdaBoostRegressor
from sklearn.dummy import DummyRegressor
from scipy import stats
from sklearn import svm
pd.options.display.float_format = '{: 3f}'.format
significanceLevel = 0.05
testSetSize = 0.25 #percentage of total set, same as paper
k = 5 # for K-fold cross-validation
lossFunction = mean_squared_error
# model = linear_model.LinearRegression()
# model  = AdaBoostRegressor()
model = DummyRegressor()
# model = svm.SVC(kernel='linear', C=1)

''' TRY USING PolynomialFeatures, cross_validate, RandomForestRegressor'''

In [3]:
# Load and clean the data
data = pd.read_csv("../data/energydata_complete.csv").dropna(how='any')
data['date'] = pd.to_datetime(data['date'])
data.set_index('date',inplace = True)
data = data.astype(np.float64)
y = data['Appliances']
X = data.iloc[:,1:]   
data.head()

,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
date,,,,,,,,,,,,,,,,,,,,,
2016-01-11 17:00:00,60.000000,30.000000,19.890000,47.596667,19.200000,44.790000,19.790000,44.730000,19.000000,45.566667,...,17.033333,45.530000,6.600000,733.500000,92.000000,7.000000,63.000000,5.300000,13.275433,13.275433
2016-01-11 17:10:00,60.000000,30.000000,19.890000,46.693333,19.200000,44.722500,19.790000,44.790000,19.000000,45.992500,...,17.066667,45.560000,6.483333,733.600000,92.000000,6.666667,59.166667,5.200000,18.606195,18.606195
2016-01-11 17:20:00,50.000000,30.000000,19.890000,46.300000,19.200000,44.626667,19.790000,44.933333,18.926667,45.890000,...,17.000000,45.500000,6.366667,733.700000,92.000000,6.333333,55.333333,5.100000,28.642668,28.642668
2016-01-11 17:30:00,50.000000,40.000000,19.890000,46.066667,19.200000,44.590000,19.790000,45.000000,18.890000,45.723333,...,17.000000,45.400000,6.250000,733.800000,92.000000,6.000000,51.500000,5.000000,45.410389,45.410389
2016-01-11 17:40:00,60.000000,40.000000,19.890000,46.333333,19.200000,44.530000,19.790000,45.000000,18.890000,45.530000,...,17.000000,45.400000,6.133333,733.900000,92.000000,5.666667,47.666667,4.900000,10.084097,10.084097


In [4]:
# Prepare the data
# taken from http://scikit-learn.org/stable/modules/cross_validation.html
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = testSetSize)

In [5]:
# We're looking for strongly correlated features to identify important ones
# the correlation threshold is arbitrarily chosen, but to use this
# we must be certain enough that this is actually the case
# So we also test against the significance level
featureSet = []
for col in data.columns:
    r,p = stats.pearsonr(data['Appliances'],data[col])
    if abs(r) > 0.08 and p < significanceLevel and col!= 'Appliances':
        print("{:<13}({: 5f},{: 5f})".format(col,r,p))
        featureSet.append(col)

lights       ( 0.197278, 0.000000)
RH_1         ( 0.086031, 0.000000)
T2           ( 0.120073, 0.000000)
T3           ( 0.085060, 0.000000)
T6           ( 0.117638, 0.000000)
RH_6         (-0.083178, 0.000000)
RH_8         (-0.094039, 0.000000)
T_out        ( 0.099155, 0.000000)
RH_out       (-0.152282, 0.000000)
Windspeed    ( 0.087122, 0.000000)


In [38]:
model.fit(X_train,y=y_train)
# mean_squared_error(y_test,model.predict(X_test))
print(cross_val_score(model,X_train,y=y_train,cv=k))


[-0.00022455 -0.0001025  -0.00028524 -0.00010675 -0.00127517]


In [40]:
mean_squared_error(y_test,model.predict(X_test))

10220.674159020522